### Fine-Tune BERT Model
Fine-tune the pre-trained BERT model on the news dataset.

In [54]:
from transformers import TFBertModel

# Load pre-trained BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased')

# Build Multi-Task Model with BERT
input_ids = tf.keras.Input(shape=(128,), dtype='int32')
attention_mask = tf.keras.Input(shape=(128,), dtype='int32')

bert_output = bert_model(input_ids, attention_mask=attention_mask)[1]  # Use pooled output

# Shared Representation
dense_1 = tf.keras.layers.Dense(64, activation="relu")(bert_output)
dropout = tf.keras.layers.Dropout(0.2)(dense_1)

# Sentiment Analysis Output
sentiment_output = tf.keras.layers.Dense(1, activation="sigmoid", name="sentiment_output")(dropout)

# Ranking Prediction Output
rank_output = tf.keras.layers.Dense(1, activation="linear", name="rank_output")(dropout)

# Define Model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=[sentiment_output, rank_output])

# Compile Model
model.compile(
    optimizer=tf.keras.optimizers.AdamW(learning_rate=2e-5),
    loss={"sentiment_output": "binary_crossentropy", "rank_output": "mse"},
    metrics={"sentiment_output": "accuracy", "rank_output": "mae"}
)

model.summary()

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 input_4 (InputLayer)        [(None, 128)]                0         []                            
                                                                                                  
 tf_bert_model_1 (TFBertMod  TFBaseModelOutputWithPooli   1094822   ['input_3[0][0]',             
 el)                         ngAndCrossAttentions(last_   40         'input_4[0][0]']             
                             hidden_state=(None, 128, 7                                           
                             68),                                                           

In [55]:
# Early Stopping
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
]

# Start MLflow Run
mlflow.set_experiment("Multi-Task Sentiment Analysis with BERT")
with mlflow.start_run():
    # Log model parameters
    mlflow.log_param("bert_model", "bert-base-uncased")
    mlflow.log_param("dropout", 0.2)
    mlflow.log_param("batch_size", 64)
    mlflow.log_param("epochs", 8)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("learning_rate", 2e-5)

    # Disable automatic logging of TensorBoard events
    mlflow.tensorflow.autolog(every_n_iter=5,log_models=True,disable=True)  

    # Train Model
    history = model.fit(
        [X_train, train_mask], 
        {"sentiment_output": y_train_sent, "rank_output": y_train_rank},
        epochs=8,
        batch_size=64,
        validation_data=([X_test, test_mask], {"sentiment_output": y_test_sent, "rank_output": y_test_rank}),
        callbacks=callbacks)

Epoch 1/8
5344/5344 [==============================] - 5229s 972ms/step - loss: 14.9801 - sentiment_output_loss: 0.4664 - rank_output_loss: 14.5137 - sentiment_output_accuracy: 0.7744 - rank_output_mae: 2.7584 - val_loss: 12.0513 - val_sentiment_output_loss: 0.3730 - val_rank_output_loss: 11.6783 - val_sentiment_output_accuracy: 0.8321 - val_rank_output_mae: 2.1905
Epoch 2/8
5344/5344 [==============================] - 5221s 977ms/step - loss: 11.0923 - sentiment_output_loss: 0.3831 - rank_output_loss: 10.7092 - sentiment_output_accuracy: 0.8293 - rank_output_mae: 2.2114 - val_loss: 11.6781 - val_sentiment_output_loss: 0.3556 - val_rank_output_loss: 11.3224 - val_sentiment_output_accuracy: 0.8443 - val_rank_output_mae: 2.1172
Epoch 3/8
5344/5344 [==============================] - 5223s 977ms/step - loss: 9.5624 - sentiment_output_loss: 0.3570 - rank_output_loss: 9.2054 - sentiment_output_accuracy: 0.8439 - rank_output_mae: 2.0241 - val_loss: 11.7430 - val_sentiment_output_loss: 0.3512 

### Evaluate Model Performance
Evaluate the model's performance on the test set using accuracy, precision, recall, and F1-score.

In [57]:
# Evaluate the model
eval_results = model.evaluate(
    [X_test, test_mask], 
    {"sentiment_output": y_test_sent, "rank_output": y_test_rank}, 
    batch_size=64
    )

1336/1336 [==============================] - 408s 305ms/step - loss: 11.6781 - sentiment_output_loss: 0.3556 - rank_output_loss: 11.3224 - sentiment_output_accuracy: 0.8443 - rank_output_mae: 2.1172
